In [ ]:
import pandas as pd 
import numpy as np 
from pathlib import Path
datafolder = Path('../test_data')

In [ ]:
import sqlite3
'https://sqlitebrowser.org/dl/'
fires_con = sqlite3.connect(datafolder / 'FPA_FOD_20170508.sqlite')
fires_cur = fires_con.cursor()
fires_cur.execute(
    'SELECT name FROM sqlite_master WHERE type="table"'
).fetchall()
fires_cur.execute(
    'PRAGMA table_info(Fires)'
).fetchall()
fires = pd.read_sql_query(
    '''SELECT STATE, FIRE_YEAR, DATETIME(DISCOVERY_DATE) AS DISCOVERY_DATE,
    FIRE_NAME, FIRE_SIZE, LATITUDE, LONGITUDE, DATETIME(CONT_DATE) AS CONTAIN_DATE
    FROM Fires''',
    fires_con
)

In [14]:
fires.columns = fires.columns.str.lower()
fires.rename(columns={'fire_size': 'acres_burned'}, inplace=True)
fires


,state,fire_year,discovery_date,fire_name,acres_burned,latitude,longitude,contain_date
0,CA,2005,2005-02-02 00:00:00,FOUNTAIN,0.10,40.036944,-121.005833,2005-02-02 00:00:00
1,CA,2004,2004-05-12 00:00:00,PIGEON,0.25,38.933056,-120.404444,2004-05-12 00:00:00
2,CA,2004,2004-05-31 00:00:00,SLACK,0.10,38.984167,-120.735556,2004-05-31 00:00:00
3,CA,2004,2004-06-28 00:00:00,DEER,0.10,38.559167,-119.913333,2004-07-03 00:00:00
4,CA,2004,2004-06-28 00:00:00,STEVENOT,0.10,38.559167,-119.933056,2004-07-03 00:00:00
...,...,...,...,...,...,...,...,...
1880460,CA,2015,2015-09-26 00:00:00,ODESSA 2,0.01,40.481637,-122.389375,2015-09-26 00:00:00
1880461,CA,2015,2015-10-05 00:00:00,None,0.20,37.617619,-120.938570,None
1880462,CA,2015,2015-05-02 00:00:00,None,0.10,37.617619,-120.938570,None
1880463,CA,2015,2015-10-14 00:00:00,None,2.00,37.672235,-120.898356,None


In [29]:
fires[['discovery_date', 'contain_date']] = fires[['discovery_date', 'contain_date']].apply(
    pd.to_datetime
)
fires['fire_month'] = fires.discovery_date.dt.month
fires

,state,fire_year,discovery_date,fire_name,acres_burned,latitude,longitude,contain_date,fire_month
0,CA,2005,2005-02-02,FOUNTAIN,0.10,40.036944,-121.005833,2005-02-02,2
1,CA,2004,2004-05-12,PIGEON,0.25,38.933056,-120.404444,2004-05-12,5
2,CA,2004,2004-05-31,SLACK,0.10,38.984167,-120.735556,2004-05-31,5
3,CA,2004,2004-06-28,DEER,0.10,38.559167,-119.913333,2004-07-03,6
4,CA,2004,2004-06-28,STEVENOT,0.10,38.559167,-119.933056,2004-07-03,6
...,...,...,...,...,...,...,...,...,...
1880460,CA,2015,2015-09-26,ODESSA 2,0.01,40.481637,-122.389375,2015-09-26,9
1880461,CA,2015,2015-10-05,None,0.20,37.617619,-120.938570,NaT,10
1880462,CA,2015,2015-05-02,None,0.10,37.617619,-120.938570,NaT,5
1880463,CA,2015,2015-10-14,None,2.00,37.672235,-120.898356,NaT,10


In [33]:
fires['days_burning'] = (fires.contain_date - fires.discovery_date).dt.days
fires.sample(5)

,state,fire_year,discovery_date,fire_name,acres_burned,latitude,longitude,contain_date,fire_month,days_burning
1276186,WI,2009,2009-04-18,TIMBERLAND TRAIL,0.44,45.428405,-90.796414,2009-04-18,4,0.0
1502715,NY,2011,2011-03-19,None,0.10,41.415466,-74.083778,2011-03-19,3,0.0
1315543,WV,2002,2002-09-12,HAMPTON OAKS LANE,0.10,39.344511,-77.987482,2002-09-12,9,0.0
1601574,UT,2012,2012-05-16,HURRICANE ASSIST 1,3.00,37.155792,-113.259042,NaT,5,NaN
1873823,CA,2009,2009-04-23,MARRA,0.50,38.391975,-122.949631,2009-04-23,4,0.0


In [46]:
fires[['state', 'days_burning']].head()
fires['mean_days'] = fires.groupby('state')['days_burning'].transform(func='mean')
fires[['state', 'days_burning', 'mean_days']].head(19)

,state,days_burning,mean_days
0,CA,0.0,1.05301
1,CA,0.0,1.05301
2,CA,0.0,1.05301
3,CA,5.0,1.05301
4,CA,5.0,1.05301
5,CA,1.0,1.05301
6,CA,1.0,1.05301
7,CA,0.0,1.05301
8,CA,0.0,1.05301
9,CA,1.0,1.05301


In [54]:
df = fires.sample(20)
df = df[['state', 'fire_year', 'fire_month']]
df.head(2)


,state,fire_year,fire_month
1028878,NC,1998,4
248261,OK,2007,11


In [62]:

from datetime import date
def func(row):
    return pd.to_datetime(date(year=row['fire_year'], month=row['fire_month'], day=1))


df[[ 'fire_year', 'fire_month']].apply(func, axis=1).head(3)

1028878   1998-04-01
248261    2007-11-01
1498601   2011-09-01
dtype: datetime64[ns]

In [66]:
fires_by_month = fires.set_index(['state', 'fire_year', 'fire_month'])
fires_by_month

discovery_date                  fire_name  \
state fire_year fire_month                                             
CA    2005      2              2005-02-02                   FOUNTAIN   
      2004      5              2004-05-12                     PIGEON   
                5              2004-05-31                      SLACK   
                6              2004-06-28                       DEER   
                6              2004-06-28                   STEVENOT   
...                                   ...                        ...   
      2015      9              2015-09-26                   ODESSA 2   
                10             2015-10-05                       None   
                5              2015-05-02                       None   
                10             2015-10-14                       None   
                3              2015-03-14  BARKER BL  BIG_BEAR_LAKE_   

                            acres_burned   latitude   longitude contain_date  \
state fire_year fire_month                                                     
CA    2005      2                   0.10  40.036944 -121.005833   2005-02-02   
      2004      5                   0.25  38.933056 -120.404444   2004-05-12   
                5                   0.10  38.984167 -120.735556   2004-05-31   
                6                   0.10  38.559167 -119.913333   2004-07-03   
                6                   0.10  38.559167 -119.933056   2004-07-03   
...                                  ...        ...         ...          ...   
      2015      9                   0.01  40.481637 -122.389375   2015-09-26   
                10                  0.20  37.617619 -120.938570          NaT   
                5                   0.10  37.617619 -120.938570          NaT   
                10                  2.00  37.672235 -120.898356          NaT   
                3                   0.10  34.263217 -116.830950          NaT   

                            days_burning  mean_days  
state fire_year fire_month                           
CA    2005      2                    0.0    1.05301  
      2004      5                    0.0    1.05301  
                5                    0.0    1.05301  
                6                    5.0    1.05301  
                6                    5.0    1.05301  
...                                  ...        ...  
      2015      9                    0.0    1.05301  
                10                   NaN    1.05301  
                5                    NaN    1.05301  
                10                   NaN    1.05301  
                3                    NaN    1.05301  

[1880465 rows x 8 columns]

In [67]:
fires_no_index = fires_by_month.reset_index(drop=False)
fires_no_index

,state,fire_year,fire_month,discovery_date,fire_name,acres_burned,latitude,longitude,contain_date,days_burning,mean_days
0,CA,2005,2,2005-02-02,FOUNTAIN,0.10,40.036944,-121.005833,2005-02-02,0.0,1.05301
1,CA,2004,5,2004-05-12,PIGEON,0.25,38.933056,-120.404444,2004-05-12,0.0,1.05301
2,CA,2004,5,2004-05-31,SLACK,0.10,38.984167,-120.735556,2004-05-31,0.0,1.05301
3,CA,2004,6,2004-06-28,DEER,0.10,38.559167,-119.913333,2004-07-03,5.0,1.05301
4,CA,2004,6,2004-06-28,STEVENOT,0.10,38.559167,-119.933056,2004-07-03,5.0,1.05301
...,...,...,...,...,...,...,...,...,...,...,...
1880460,CA,2015,9,2015-09-26,ODESSA 2,0.01,40.481637,-122.389375,2015-09-26,0.0,1.05301
1880461,CA,2015,10,2015-10-05,None,0.20,37.617619,-120.938570,NaT,NaN,1.05301
1880462,CA,2015,5,2015-05-02,None,0.10,37.617619,-120.938570,NaT,NaN,1.05301
1880463,CA,2015,10,2015-10-14,None,2.00,37.672235,-120.898356,NaT,NaN,1.05301


In [81]:
fires_by_year = fires.groupby(['state', 'fire_year'])[['acres_burned']].sum()
fires_by_year.unstack()

acres_burned                                                  \
fire_year         1992        1993        1994        1995        1996   
state                                                                    
AK          142717.400  687191.500  261964.900   43962.100  598797.200   
AL            1253.200    2336.800   34664.400   38896.600   47683.200   
AR            1478.500    1760.200    2303.900    5190.400    5286.800   
AZ           42593.650  205567.020  224199.735  262152.830  187400.565   
CA          296439.800  321495.300  407944.700  216068.700  707110.500   
CO            7451.900   35555.300   70760.600   20043.700   70040.500   
CT             466.050     401.100     943.150     427.400      66.450   
DC               0.400       9.800       5.100       0.600         NaN   
DE             124.000       1.000      96.000    2014.400         NaN   
FL           95279.700   79755.200  103085.400   56682.700  107511.500   
GA           27016.370   40515.590   25013.890   24460.520   37975.890   
HI           11161.300    2562.000      78.600    1027.500     380.300   
IA              85.300      30.700     105.400      57.400    1026.400   
ID          684658.900    7978.200  729029.600  268681.300  896659.900   
IL             235.400      29.800     383.700     993.200     149.800   
IN             247.300     464.300     577.900     881.400     495.000   
KS             251.600    1809.300   84460.100    4537.900  147570.400   
KY             604.500     843.100    4660.600    5863.600    1117.800   
LA           25129.100    8066.800    2520.600    7277.900    8254.100   
MA               5.400       1.500       0.100       8.700       2.200   
MD            1160.300    1672.900     786.900    1304.400     312.100   
ME            4399.300    1413.150    2012.690    1137.080     403.350   
MI            2987.400    1008.000    6704.200    4872.300    2921.700   
MN           30182.130   19087.710   37150.320   46067.310   29633.940   
MO           28716.000   14997.300   36249.400   50196.400    8538.300   
MS           60830.100   54108.300   54478.500   68013.800  101707.700   
MT           38591.720   12598.410  265689.350   25183.950  208840.450   
NC           27489.100   26461.200   44936.200   23404.300   17358.000   
ND            6373.900    3923.900   11743.300    2047.600    4801.800   
NE            1227.100     173.500    6869.500   55757.800    8278.300   
NH             164.671     166.663     207.234     309.042      95.734   
NJ           16603.650     922.700    2725.850   22962.146     929.600   
NM           99202.450  302344.040  416646.720  139862.350  171645.960   
NV           92345.800   53979.900  196057.800  120031.400  627484.000   
NY            2334.400    1126.300    1154.600    9167.450     417.300   
OH             101.700      42.700     125.700     292.300      70.300   
OK           47575.100   69803.500   75164.200   94478.900  207500.000   
OR          117437.360   13843.280  276054.320   61282.490  590641.840   
PA              59.700      40.800      66.500      51.200      18.900   
PR             127.000     163.000    1000.000         NaN     502.000   
RI                 NaN         NaN      25.000         NaN         NaN   
SC           31202.800   34284.700   30354.100   17794.800   28609.600   
SD           59412.280    6133.300   16520.050   10273.220   18689.020   
TN             429.600     866.400    3300.200    2049.200     484.100   
TX           33610.300  116873.700   70917.500   56371.100  216223.000   
UT           37531.910   19887.470  213555.100  199115.640  511165.790   
VA             434.600     564.000    1205.500   15136.750    3259.500   
VT             270.000       3.000      25.300       1.200         NaN   
WA           89681.050   22629.000  271438.100    6727.800  148134.100   
WI            2520.610    1445.220    5242.020    2316.930    2927.970   
WV            7514.400    9434.000   50906.800   41583.600    5858.500   
WY           2